# Active learning con modAL
Un questo notebook usiamo la libreria modAL per effettuare active learning sul modello creato durante la prima sessione

In [42]:
from modAL.models import ActiveLearner
from modAL.uncertainty import entropy_sampling, uncertainty_sampling, margin_sampling
from sklearn.ensemble import RandomForestClassifier
import joblib
import pandas as pd
import numpy as np


## Carichiamo dati e modello dalla prima sessione

X = pd.read_csv('../data/adult_training.csv')
y = X['income'].values
X = X.drop(['income'],axis=1)

X_pool = pd.read_csv('../data/adult_test.csv')
y_pool = X_pool['income'].values
X_pool = X_pool.drop(['income'],axis=1)

clf = joblib.load('../model.pkl')

Utilizziamo la classe ActiveLearner dalla libreria modAL come wrapper per il nostro modello allenato.
L'istanza learner ha la stessa struttura di un sklearn BaseEstimator (ha ad esempio un metodo predict_proba)

In [43]:
# initializing the learner
learner = ActiveLearner(
    estimator=clf,
    X_training=X, 
    y_training=y,
    query_strategy=uncertainty_sampling
)

Qui di seguito assumiamo di avere un pool di dati non etichettati X_pool. In realta' le etichette le conosciamo ma facciamo finta che y_pool sia la risposta di quello che abbiamo definito come 'oracolo'.

In [44]:
# query for labels
n_queries = 10
for idx in range(n_queries):
    # Chiediamo al modulo active learning quali dati etichettare
    query_idx, query_inst = learner.query(X_pool)
    # Alleniamo il classificatore con il nuovo dato una volta che abbiamo ottenuto l'etichetta corrispondende
    # dall'oracolo
    oracle_answer = y_pool[query_idx[0]:query_idx[0]+1]
    learner.teach(X_pool.iloc[query_idx[0]:query_idx[0]+1], oracle_answer)
    
    # Rimuoviamo i dati etichettati dal pool dei dati da etichettare
    X_pool = X_pool.drop(query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)    
    print(learner.predict_proba(query_inst))    
    print(query_inst.iloc[0])



[[0.50004453 0.49995547]]
age                          33
work_class              Private
education             Bachelors
marital_status          Married
occupation         Professional
relationship            Husband
race                      White
sex                        Male
capital_gain                  0
capital_loss                  0
hours_per_week               50
native_country    United-States
Name: 1710, dtype: object
[[0.49993331 0.50006669]]
age                          32
work_class              Private
education             Bachelors
marital_status          Married
occupation         Professional
relationship            Husband
race                      White
sex                        Male
capital_gain                  0
capital_loss                  0
hours_per_week               50
native_country    United-States
Name: 2145, dtype: object
[[0.49981646 0.50018354]]
age                          32
work_class              Private
education             Bachelors
marita